# Neutronics simulation with CAD geometry

This example creates a CAD geometry and then carries out a neutronics simulation with a heating cell tally.

This section makes a few components and places them in a reactor object. These will form the CAD model with which we will perform a neutronics simulation.

In [1]:
import paramak

plasma = paramak.Plasma(
    minor_radius=150.,
    major_radius=450.,
    triangularity=0.55,
    elongation=2.,
    rotation_angle=360
)

blanket = paramak.BlanketFP(
    plasma=plasma,
    thickness=20,
    stop_angle=90,
    start_angle=-90,
    offset_from_plasma=40,
    rotation_angle=360,
)

my_reactor = paramak.Reactor([plasma, blanket])
my_reactor.export_stl(['plasma.stl', 'blanket.stl'])
my_reactor.show()

Overwriting auto display for cadquery Workplane and Shape


This section forms the neutronics model by combining the 3D model, the plasma source and some assigned materials. Additionally, the tallies to record the heating are specified. There are a few different methods of converting a CAD model into a neutronics model. Each method has pros and cons.

The rector can also be converted from stl or stp to h5m format which is the 3D geometry format used by DAGMC to represent neutronics geometry.

stl-to-h5m can convert stl files to a h5m geometry
https://github.com/fusion-energy/stl_to_h5m

cad-to-h5m can convert stp files to h5m geometry
https://github.com/fusion-energy/cad_to_h5m

In [3]:
from stl_to_h5m import stl_to_h5m

stl_to_h5m(
    files_with_tags=[
        ('plasma.stl', 'plasma'),
        ('blanket.stl', 'blanket')
    ],
    h5m_filename='dagmc.h5m',
)

'dagmc.h5m'

The model can now be simulated in OpenMC. OpenMC is imported along with some convenience packages

In [4]:
import openmc
import openmc_dagmc_wrapper as odw
import openmc_plasma_source as ops

The following code makes use of the previously created neutronics geometry (h5m file) and assigns actual materials to the material tags. Sets simulation intensity and specifies the neutronics results to record (know as tallies).

In [6]:
# this links the material tags in the dagmc h5m file with materials.
# these materials are input as strings so they will be looked up in the
# neutronics material maker package
material_tag_to_material_dict = {
    'plasma':'DT_plasma',
    'blanket':'Li4SiO4',
}

geometry = odw.Geometry(h5m_filename='dagmc.h5m')

materials = odw.Materials(
    h5m_filename='dagmc.h5m',
    correspondence_dict=material_tag_to_material_dict
)

tally1 = odw.CellTally(
    tally_type="TBR",
    target="blanket",
    materials=materials
)

tallies = openmc.Tallies([tally1])

settings = odw.FusionSettings()
settings.batches = 1
settings.particles = 100
# assigns a ring source of DT energy neutrons to the source using the
# openmc_plasma_source package
settings.source = ops.FusionRingSource(fuel="DT", radius=350)


my_model = openmc.Model(
    materials=materials, geometry=geometry, settings=settings, tallies=tallies
)
statepoint_file = my_model.run()

Using the DOUBLE-DOWN interface to Embree.
Loading file dagmc.h5m
Initializing the GeomQueryTool...
Using faceting tolerance: 1.81323e-317
Building acceleration data structures...
Implicit Complement assumed to be Vacuum


          particles and rapid simulation failure.
/opt/openmc/openmc/mixin.py:67: IDWarning: Another Surface instance already exists with id=9999.
  warn(msg, IDWarning)
/opt/openmc/openmc/mixin.py:67: IDWarning: Another Cell instance already exists with id=9999.
  warn(msg, IDWarning)


                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%%%%%%%%%%%%%%%%%%
                #####################     %%%%%%%%%%%%%%%%%%%%%
                ######################     %%%%%%%%%%%%%%%%%%%%
                #######################     %%%%%%%%%%%%%%%%%%
                 #######################     %%%%%%%%%%%%%%%%%
                 #####################

Extracting the resulting tally value from the statepoint file

In [17]:
sp = openmc.StatePoint(statepoint_file)

tbr_tally = sp.get_tally(name='blanket_TBR')

df = tbr_tally.get_pandas_dataframe()

tbr_tally_result

/opt/openmc/openmc/tallies.py:278: RuntimeWarning: invalid value encountered in true_divide
  self._std_dev[nonzero] = np.sqrt((self.sum_sq[nonzero]/n -


0    0.282663
Name: mean, dtype: float64

**Learning Outcomes from Task 11**

- Neutronics simulations can be performed by combining CAD models, neutron sources and material definitions.